# Solution - Learning Satellite orbit from data through  Perturbation forces (in RSW) - by PySINDy

In [40]:
import pysindy as ps
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from src.utils import *
from src.propagation import *
import pickle

Load the CSV file with correct column extraction<br>
Assuming the first column is time, next three are position (r1, r2, r3), last three are velocity (v1, v2, v3)

In [41]:
df = pd.read_csv('./data/Challenge1.csv', header=0, usecols=[0,1,2,3,4,5,6], names=['time','r1', 'r2', 'r3', 'v1', 'v2', 'v3'])

Extract position and velocity arrays<br>
Convert time strings to pandas datetime

In [42]:
t_raw = pd.to_datetime(df['time'])
# Calculate seconds relative to the first time point
t = (t_raw - t_raw.iloc[0]).dt.total_seconds().values # Time in seconds relative to first time point
r = df[['r1', 'r2', 'r3']].values
v = df[['v1', 'v2', 'v3']].values

Copying in parameters (from `src/parameter.csv`)

In [43]:
mu = 3.986004414498200e14  # Central Body's gravitational constant (m^3/s^2) - from parameter.csv
Re = 6.378136460000000e6  # Central Body's equatorial radius (m) - from parameter.csv

**Here is an important step**:
To help the algorithm learn better, it is often a good idea to non-dimensionalise / rescale your data such that it is at order $\mathcal{O}(1)$. 

Alternatively, you may want to use PySR in-built dimension tool to aid the learning.

In [44]:
## Finding the typical time scale and length scale of the data
T_dim = np.sqrt(Re**3 / mu)  # Time unit for non-dimensionalization (s)
L_dim = Re  # Length unit for non-dimensionalization (m)

Convert the r,v representation to equinoctial

In [45]:
equi = ijk2equinoctial(r,v,mu)

 **Non-dimensionalize equinoctial elements** (see comment above)

In [46]:
equi[..., 0] = equi[..., 0] / L_dim  # Only the first element of equinoctial elements has the dimension of length. 
# The rest are dimensionless
t = t / T_dim  # Non-dimensional time

 Setting up a 2nd order finite difference matrix

In [47]:
from scipy.sparse import spdiags

In [48]:
N_pt = equi.shape[0]  # Number of points in the time series
dt = t[1] - t[0]
I  = spdiags(np.array([[0],[0],[0],[0],[0], [1],[0],[0],[0],[0],[0]]) @ np.ones((1,N_pt)) , np.array([0,1,2,3,4,5,6,7,8,9,10]), m = N_pt - 10, n = N_pt)
D1 = spdiags(np.array([[-1/1260], [5/504], [-5/84], [5/21], [-5/6], [0], [5/6], [-5/21], [5/84], [-5/504], [1/1260]]) @ np.ones((1,N_pt)) / dt, np.array([0,1,2,3,4,5,6,7,8,9,10]), m = N_pt - 10, n = N_pt)

 Applying to equinoctial elements

In [49]:
d_equi_dt = D1 @ equi # Time derivative of equinoctial elements 
equi_ = I @ equi # The equinoctial elements value at the time point of the derivative

The eqn. to transform  FR FS and FW into d (element)/ dt can be written as:

    d (element)/ dt = A(element,mu) * F_RSW(element) + b(element,mu)

The function A and b can be found in propagation.py
(hint: look at EquinoctialPropagation.ipynb or KeplerianPropagation.ipynb to see how the derivatives are computed from FR FS and FW)!

In [50]:
A_ = np.stack([RSW2equi_A(e, 1) for e in equi_])
b_ = np.stack([RSW2equi_b(e, 1) for e in equi_])

In [51]:
F_RSW_equi = np.array([
    # np.linalg.pinv(A_[i]) @ (d_equi_dt[i] - b_[i])
    np.linalg.lstsq(A_[i][:-1,...], d_equi_dt[i][:-1])[0]  # More efficient and stable than pinv
    for i in range(A_.shape[0])
])

In [52]:
print("F_RSW_equi shape:", F_RSW_equi.shape) 
print("equi_ shape:", equi_.shape)
print("d_equi_dt shape:", d_equi_dt.shape)

F_RSW_equi shape: (17271, 3)
equi_ shape: (17271, 6)
d_equi_dt shape: (17271, 6)


In [53]:
FR_tar = F_RSW_equi[:,0] 
FS_tar = F_RSW_equi[:,1]
FW_tar = F_RSW_equi[:,2]
RHS_in = equi_ 

Calculate extra inputs from equi_ (midpoint equinoctial elements)

In [54]:
p_ = equi_[:, 0]        
f_ = equi_[:, 1]        
g_ = equi_[:, 2]        
h_ = equi_[:, 3]        
k_ = equi_[:, 4]       
L_ = equi_[:, 5]        

While PySR can learn directly the formula for $F_R$, $F_S$, $F_W$ as function of ($p$,$f$,$g$,$h$,$k$,$L$), it is often more helpful to incorperate prior knowledge to aid the learning. 

Below codify a few functions of ($p$,$f$,$g$,$h$,$k$,$L$) that is going to be helpful.

In [55]:
s_ = 1 + h_**2 + k_**2  # s = 1 + h^2 + k^2 = sec^2(i/2)
w_ = 1 + f_ * np.cos(L_) + g_ * np.sin(L_)  # w = 1 + f*cos(L) + g*sin(L)
r_val_ = p_ / w_         # r = p / w

In [56]:
sin_u_sin_i_ = 2 * (h_ * np.sin(L_) - k_ * np.cos(L_)) / s_      # sin(u)*sin(i)
cos_u_sin_i_ = 2 * (h_ * np.cos(L_) + k_ * np.sin(L_)) / s_      # cos(u)*sin(i)
cos_i_ = (1 - h_**2 - k_**2) / s_                                # cos(i)

Stack all features together for a custom SINDy library

In [57]:
RHS_in = np.column_stack([
    r_val_**(-4), sin_u_sin_i_,
    cos_u_sin_i_, cos_i_
])
# You may also want to try a larger feature set, e.g.,
# RHS_in = np.column_stack([p_, f_, g_, h_, k_, L_, s_, w_, r_val_, sin_u_sin_i_, cos_u_sin_i_, cos_i_, mu, Re])

### Applying PySINDy
To make PySINDy learn based on `F_RSW_equi` instead of the typical time derivative, and to make use of the custom library `RHS_in`, we need to do a bit of hack.
`F_RSW_equi` can replace the `x_dot` argument, which normally takes the time derivatives as the learning target.
Meanwhile, we can exploit the `u` argument in SINDy, which usually take external time-varying control signal as part of the learning problem. Here, we highjack this functionality to input the custom library `RHS_in`. SINDy will then generate a polynomial library of these "control signal" with the state variables `x`. 

Furthermore, since we no longer use `x`, we can replace it with a placeholder. To prevent it from being learnt as a useful signal, we will use a large random vector as the placeholder. Note that it needs to be of the same dimensions as `F_RSW_equi`.

In [ ]:
x_placeholder = np.random.random((F_RSW_equi.shape[0],3))*1000  # Large random placeholder to prevent SINDy from learning useful signal from x

Now, we can apply PySINDy

In [ ]:
opt = ps.STLSQ(threshold=1,normalize_columns=True) # You may want to tune the threshold value to see how it affects the sparsity of the learnt model
model = ps.SINDy(feature_library=ps.PolynomialLibrary(degree = 3,include_bias=False), # We use polynomial library to generate combinations of the input features. This is contain polynomials columns of RHS_in and x_placeholder up to the specified degree. You may want to try other libraries as well.
                 optimizer=opt) 
model.fit(x_placeholder, u=RHS_in,t=dt, x_dot=F_RSW_equi) # Fit the model. Note that we are using x_placeholder as x, RHS_in as u, and F_RSW_equi as x_dot
model.print()

(x0)' = 1.500 u0 u1^2 + -0.750 u0 u2^2 + -0.750 u0 u3^2
(x1)' = -1.500 u0 u1 u2
(x2)' = -1.500 u0 u1 u3


Reading from the model learnt above, we have the following formula as the output

$$ F_R = 0.75 \frac{2(\sin{(u)}\sin{(i)})^2 -  (\cos{(u)}\sin{(i)})^2 -  \cos^2{(i)}}{r^4} = 0.75 \frac{8 (\sin{L} - k \cos{L})^2 -  4 (h \cos{L} + k \sin{L})^2 -  (1-h^2-k^2)^2}{s^2 r^4}  $$
$$ F_S = -1.5 \frac{\sin{(u)}\sin{(i)}\cos{(u)}\sin{(i)}}{r^4} = -6\frac{(\sin{L} - k \cos{L})(h \cos{L} + k \sin{L})}{s^2 r^4}$$
$$ F_W =  -1.5 \frac{\sin{(u)}\sin{(i)} \cos^2{(i)}}{r^4}=-3 \frac{(\sin{L} - k \cos{L})(1-h^2-k^2)}{s^2 r^4}$$

The slight downside of hacking PySINDy this way is that we can no longer exploit their in-built propagator.
Instead, to propagate the model and compare the result with the correct answer, we can open the above saved models in the `EquinoctialPropagation_PySINDy.ipynb` and propagate the learnt model.